# Denoising Pipeline Development

In [ ]:
%load_ext autoreload
%autoreload 2

import fibsem
import salami
from fibsem import utils, acquire, alignment, calibration, milling
from fibsem.structures import BeamType, FibsemPatternSettings, FibsemMillingSettings, FibsemPattern
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import logging

from salami.structures import SalamiSettings, run_salami, create_sweep_parameters



In [ ]:
microscope, settings = utils.setup_session()


In [ ]:
eb_image, ib_image = acquire.take_reference_images(microscope, settings.image)

fig, ax = plt.subplots(1,2, figsize=(10,5))
ax[0].imshow(eb_image.data, cmap="gray")
ax[1].imshow(ib_image.data, cmap="gray")
plt.show()
